In [1]:

import MetaTrader5 as mt5
import pandas as pd 
import threading as th
import time
from datetime import datetime

import math
import pytz

import plotly.express as px

In [ ]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# دریافت داده‌های تاریخی از MetaTrader 5
def get_data(symbol, timeframe, n=10000):
    mt5.initialize()
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df

# پیش‌پردازش داده‌ها و ایجاد ویژگی‌ها
def preprocess_data(df):
    df['high_low'] = df['high'] - df['low']
    df['open_close'] = df['open'] - df['close']
    df['ma5'] = df['close'].rolling(window=5).mean()
    df['ma10'] = df['close'].rolling(window=10).mean()
    df['std5'] = df['close'].rolling(window=5).std()
    df['std10'] = df['close'].rolling(window=10).std()
    df.dropna(inplace=True)
    return df

# پارامترها
symbol = 'GBPUSD_o'
timeframe = mt5.TIMEFRAME_M5
df = get_data(symbol, timeframe)

# پیش‌پردازش داده‌ها
df = preprocess_data(df)

# جدا کردن ویژگی‌ها و هدف
X = df[['open', 'high', 'low', 'close', 'ma5', 'ma10', 'std5', 'std10']]
y = df['high_low']

# مقیاس‌بندی داده‌ها
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# تقسیم داده‌ها به مجموعه‌های آموزشی و آزمایشی
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ایجاد مدل‌ها
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}

# تنظیم هایپارامترها با استفاده از GridSearchCV
param_grid = {
    'Ridge': {'alpha': [0.1, 1, 10, 100]},
    'Lasso': {'alpha': [0.1, 1, 10, 100]},
    'ElasticNet': {'alpha': [0.1, 1, 10, 100], 'l1_ratio': [0.1, 0.5, 0.9]}
}

best_models = {}
for name, model in models.items():
    if name in param_grid:
        grid_search = GridSearchCV(model, param_grid[name], cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X_train, y_train)
        best_models[name] = grid_search.best_estimator_
        print(f"Best parameters for {name}: {grid_search.best_params_}")
    else:
        model.fit(X_train, y_train)
        best_models[name] = model

# ارزیابی مدل‌ها
for name, model in best_models.items():
    # استفاده از Cross-Validation برای ارزیابی
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mean = np.mean(np.sqrt(-cv_scores))
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'{name} - Cross-Validation RMSE: {cv_mean}, Test MSE: {mse}, R2 Score: {r2}')
    plt.figure(figsize=(10, 6))
    plt.plot(y_test.values, label='Actual')
    plt.plot(y_pred, label='Predicted')
    plt.title(name)
    plt.legend()
    plt.show()

# پیش‌بینی اندازه کندل بعدی با بهترین مدل
best_model = best_models['Ridge']
latest_data = X_scaled[-1].reshape(1, -1)
next_candle_size = best_model.predict(latest_data)
print(f'Next Candle Size: {next_candle_size[0]}')


In [89]:
df = pd.read_csv('trade_data.csv')
col = df.pop('balance')
df.insert(0, col.name, col)
df.to_csv('trade_data.csv', index=False)

In [44]:
def calculate_volume(losses, distance, reward_ratio):
    return (losses + distance * reward_ratio) / distance

# پارامترهای اولیه
initial_volume = 1
stop_loss_pips = 50
reward_ratio = 3  # نسبت سود به ضرر

# لیست برای ذخیره حجم معاملات و ضررها
volumes = [initial_volume]
losses = 0

# شبیه‌سازی چندین مرحله از معاملات
for i in range(10):
    # محاسبه حجم معامله جدید
    new_volume = calculate_volume(losses, stop_loss_pips, reward_ratio)
    volumes.append(new_volume)
    
    # فرض کنید معامله جدید ضرر کند
    losses += new_volume * stop_loss_pips
    
    print(f"Stage {i+1}: Volume = {new_volume}, Total Losses = {losses}")

# نمایش حجم معاملات
print(volumes)




Stage 1: Volume = 3.0, Total Losses = 150.0
Stage 2: Volume = 6.0, Total Losses = 450.0
Stage 3: Volume = 12.0, Total Losses = 1050.0
Stage 4: Volume = 24.0, Total Losses = 2250.0
Stage 5: Volume = 48.0, Total Losses = 4650.0
Stage 6: Volume = 96.0, Total Losses = 9450.0
Stage 7: Volume = 192.0, Total Losses = 19050.0
Stage 8: Volume = 384.0, Total Losses = 38250.0
Stage 9: Volume = 768.0, Total Losses = 76650.0
Stage 10: Volume = 1536.0, Total Losses = 153450.0
[1, 3.0, 6.0, 12.0, 24.0, 48.0, 96.0, 192.0, 384.0, 768.0, 1536.0]


In [66]:
mt5.initialize(path = r"C:\Program Files\LiteFinance MT5 2\terminal64.exe")
mt5.login(89373537 , password= 'Mahdi1400@' , server= 'LiteFinance-MT5-Demo')

True

In [2]:
def init():
    # mt5.initialize(path = r"C:\Program Files\LiteFinance MT5 real\terminal64.exe")
    # mt5.login(6910350 , password= 'Mahdi1400@' , server= 'LiteFinance-MT5-Live')
    mt5.initialize(path = r"C:\Program Files\LiteFinance MT5 real 2\terminal64.exe")
    mt5.login(6920086 , password= 'Mahdi1400@' , server= 'LiteFinance-MT5-Live')
init()

In [1]:
if -20 > -30 :
    print('hi')

hi


In [11]:
pos = mt5.positions_get( )
# pos = mt5.positions_get( ticket =  29091502 )[0].profit
jam = 0
tick = {}
for po in pos:
    tick[po.ticket] = po.profit
    jam = jam + po.profit

for i in tick:
    print(i)

jam

5119949446
5119949450


-203.79999999999995

In [160]:
def al():
    pos = mt5.ORDER_TYPE_SELL


    tp = abs(mt5.symbol_info_tick(symbol).ask - 0.0005)  
    sl = abs(mt5.symbol_info_tick(symbol).bid + 0.005) 

    lot = 0.1
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 100
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type":  pos ,
        "price": price,
        "deviation": deviation,
        "magic": 234002,
        "comment": "Gu",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK ,
    }
        
    # send a trading request
    result = mt5.order_send(request)
    if result.retcode == 10009 :
        return result

result = al()

pos = mt5.positions_get( ticket =  result.order )[0].profit
pos

# if result.retcode == 10009 :
#     print(result)

0.0

In [168]:
# نرخ‌های جفت ارزها
gbpusd_rate = 1.273
eurusd_rate = 1.0846

# محاسبه نسبت GBP به EUR
gbp_to_eur_ratio = gbpusd_rate / eurusd_rate

# مقدار خرید کل به دلار آمریکا
total_amount_usd = 1000

# محاسبه مقدار خرید هر کدام
amount_gbpusd = total_amount_usd / (gbp_to_eur_ratio + 1)
amount_eurusd = gbp_to_eur_ratio * total_amount_usd / (gbp_to_eur_ratio + 1)

# محاسبه حجم معامله به لات
lot_size_gbpusd = amount_gbpusd / (gbpusd_rate * 100000)
lot_size_eurusd = amount_eurusd / (eurusd_rate * 100000)

print(f"حجم معامله GBPUSD: {lot_size_gbpusd:.5f} لات")
print(f"حجم معامله EURUSD: {lot_size_eurusd:.5f} لات")


حجم معامله GBPUSD: 0.00361 لات
حجم معامله EURUSD: 0.00498 لات


In [167]:
pos = mt5.positions_get( ticket=29095575)
pos

(TradePosition(ticket=29095575, time=1717188910, time_msc=1717188910307, time_update=1717188910, time_update_msc=1717188910307, type=0, magic=234002, identifier=29095575, reason=3, volume=1.0, price_open=1.27255, sl=0.0, tp=0.0, price_current=1.27274, swap=0.0, profit=19.0, symbol='GBPUSD_o', comment='one', external_id=''),)

In [22]:
rob , rob = 15  , 30

In [23]:
rob

30

In [87]:
v = 0.1

for i in range(15):
    if i <= 4 :
        v = v + (v / 2 )
    

    else:
        v = v + (v / 3 )
    print(v)

0.15000000000000002
0.22500000000000003
0.3375
0.5062500000000001
0.7593750000000001
1.0125000000000002
1.3500000000000003
1.8000000000000005
2.400000000000001
3.200000000000001
4.266666666666668
5.688888888888891
7.585185185185188
10.113580246913584
13.484773662551445


In [73]:
v = 0.1

for i in range(15):
    if i <= 3:
        v = v + (v / 2)
    else : 
        v = v + (v / 4)
    
    print(v)

0.15000000000000002
0.22500000000000003
0.3375
0.5062500000000001
0.6328125000000001
0.7910156250000001
0.9887695312500001
1.2359619140625002
1.5449523925781252
1.9311904907226565
2.4139881134033208
3.017485141754151
3.7718564271926884
4.714820533990861
5.893525667488576


In [73]:

positions = mt5.positions_get()

positions


# def close(symbol):
    
    
#     positions = mt5.positions_get()
#     def close_position(position ):

#         tick = mt5.symbol_info_tick(position.symbol)

#         request = {
#             "action": mt5.TRADE_ACTION_DEAL,
#             "position": position.ticket,
#             "symbol": position.symbol,
#             "volume": position.volume,
#             "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
#             "price": tick.ask if position.type == 1 else tick.bid,  
#             "deviation": 100,
#             "magic": 234002,
#             "comment": "Gu",
#             "type_time": mt5.ORDER_TIME_GTC,
#             "type_filling": mt5.ORDER_FILLING_FOK ,
#         }

#         result = mt5.order_send(request)
#         return result

#     for position in positions:
#         print(position)
#         time.sleep(0.01)
#         if position.symbol == symbol :
#             close_position(position )


# close('AUDUSD_o')

()

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np

# Initialize MetaTrader 5 connection
if not mt5.initialize():
    print("MetaTrader5 initialization failed")


def get_data(symbol, timeframe, n=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s').dt.tz_localize('UTC').dt.tz_convert(tehran_timezone)
    df['time'] = df['time'].dt.tz_localize(None)
    df.set_index('time', inplace=True)
    return df

# Function to calculate Bollinger Bands
def bollinger_bands(price, window=20, num_of_std=2):
    rolling_mean = price.rolling(window).mean()
    rolling_std = price.rolling(window).std()
    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)
    return upper_band, rolling_mean, lower_band

# Function to calculate RSI
def RSI(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to calculate MACD
def MACD(price, slow=26, fast=12, signal=9):
    exp1 = price.ewm(span=fast, adjust=False).mean()
    exp2 = price.ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    hist = macd - signal_line
    return macd, signal_line, hist

# Function to calculate indicators
def calculate_indicators(df):
    df['upper_band'], df['middle_band'], df['lower_band'] = bollinger_bands(df['close'])
    df['rsi'] = RSI(df['close'])
    df['macd'], df['signal'], df['hist'] = MACD(df['close'])
    df['sma_short'] = df['close'].rolling(window=5).mean()
    df['sma_long'] = df['close'].rolling(window=20).mean()
    return df

# Function to implement the strategy
def scalping_strategy(symbol):
    df = get_data(symbol, mt5.TIMEFRAME_M1)
    df = calculate_indicators(df)

    for i in range(1, len(df)):
        buy_signal = (df['close'][i] > df['lower_band'][i]) and (df['rsi'][i] > 30) and (df['macd'][i] > df['signal'][i]) and (df['sma_short'][i] > df['sma_long'][i])
        sell_signal = (df['close'][i] < df['upper_band'][i]) and (df['rsi'][i] < 70) and (df['macd'][i] < df['signal'][i]) and (df['sma_short'][i] < df['sma_long'][i])

        if buy_signal:
            print(f"Buy signal at {df.index[i]} - Price: {df['close'][i]}")
            # Place a buy order here
        elif sell_signal:
            print(f"Sell signal at {df.index[i]} - Price: {df['close'][i]}")
            # Place a sell order here

# Call the strategy function for a specific symbol

symbol = 'GBPUSD_o'
while True:
    try:
        scalping_strategy(symbol)
        time.sleep(60)  # Wait for 1 minute before checking again
    except Exception as e:
        print(f"Error: {e}")
        time.sleep(60)  # Wait for 1 minute before retrying

# Shutdown MetaTrader 5 connection



In [ ]:
import pytz
import pandas as pd

tehran_timezone = pytz.timezone('Asia/Tehran')
now = datetime.now(tehran_timezone)


rates = mt5.copy_rates_from_pos('GBPUSD_o', mt5.TIMEFRAME_M1, 0, 100)
df = pd.DataFrame(rates)
df['time'] = pd.to_datetime(df['time'], unit='s').dt.tz_localize('UTC').dt.tz_convert(tehran_timezone)
df['time'] = df['time'].dt.tz_localize(None)
df.set_index('time', inplace=True)
df

In [12]:
for s in range(1, 10):
    if s == 5:
        break
print(s)

5


In [8]:


utc_from = datetime(2024, 5, 21, hour = 8, tzinfo=tehran_timezone)
utc_to = datetime(2024 , 5, 23 , hour = 21, tzinfo=tehran_timezone)
# get bars from USDJPY M5 within the interval of 2020.01.10 00:00 - 2020.01.11 13:00 in UTC time zone
ticks = mt5.copy_ticks_range('GBPUSD_o', utc_from, utc_to, mt5.COPY_TICKS_ALL)
# create DataFrame out of the obtained data
ticks_frame = pd.DataFrame(ticks)
ticks_frame['time'] = pd.to_datetime(ticks_frame['time'], unit='s').dt.tz_localize('UTC').dt.tz_convert(tehran_timezone)
ticks_frame['time'] = ticks_frame['time'].dt.tz_localize(None)
ticks_frame

,time,bid,ask,last,volume,time_msc,flags,volume_real
0,2024-05-21 08:04:00,1.27066,1.27066,0.0,0,1716266040191,130,0.0
1,2024-05-21 08:04:09,1.27064,1.27064,0.0,0,1716266049111,134,0.0
2,2024-05-21 08:04:09,1.27065,1.27065,0.0,0,1716266049206,134,0.0
3,2024-05-21 08:04:09,1.27066,1.27066,0.0,0,1716266049303,134,0.0
4,2024-05-21 08:04:09,1.27065,1.27065,0.0,0,1716266049403,134,0.0
...,...,...,...,...,...,...,...,...
187994,2024-05-23 21:03:58,1.27134,1.27134,0.0,0,1716485638483,134,0.0
187995,2024-05-23 21:03:58,1.27135,1.27135,0.0,0,1716485638887,134,0.0
187996,2024-05-23 21:03:58,1.27138,1.27138,0.0,0,1716485638985,134,0.0
187997,2024-05-23 21:03:59,1.27139,1.27139,0.0,0,1716485639284,134,0.0


In [9]:
import pytz
import pandas as pd

tehran_timezone = pytz.timezone('Asia/Tehran')
now = datetime.now(tehran_timezone)




# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
utc_from = datetime(2024, 5, 17, hour = 8, tzinfo=tehran_timezone)
utc_to = datetime(2024 , 5, 20 , hour = 21, tzinfo=tehran_timezone)
# get bars from USDJPY M5 within the interval of 2020.01.10 00:00 - 2020.01.11 13:00 in UTC time zone
ticks = mt5.copy_ticks_range("GBPUSD_o", utc_from, utc_to, mt5.COPY_TICKS_ALL)

 

 
# create DataFrame out of the obtained data
ticks_frame = pd.DataFrame(ticks)
ticks_frame['time'] = pd.to_datetime(ticks_frame['time'], unit='s').dt.tz_localize('UTC').dt.tz_convert(tehran_timezone)
ticks_frame['time'] = ticks_frame['time'].dt.tz_localize(None)
ticks_frame
 

,time,bid,ask,last,volume,time_msc,flags,volume_real
0,2024-05-17 08:04:01,1.26598,1.26598,0.0,0,1715920441320,130,0.0
1,2024-05-17 08:04:03,1.26597,1.26597,0.0,0,1715920443523,134,0.0
2,2024-05-17 08:04:03,1.26595,1.26595,0.0,0,1715920443724,134,0.0
3,2024-05-17 08:04:05,1.26596,1.26596,0.0,0,1715920445326,134,0.0
4,2024-05-17 08:04:05,1.26597,1.26597,0.0,0,1715920445919,134,0.0
...,...,...,...,...,...,...,...,...
89752,2024-05-20 21:03:58,1.27114,1.27114,0.0,0,1716226438812,134,0.0
89753,2024-05-20 21:03:59,1.27111,1.27111,0.0,0,1716226439334,134,0.0
89754,2024-05-20 21:03:59,1.27110,1.27110,0.0,0,1716226439412,134,0.0
89755,2024-05-20 21:03:59,1.27111,1.27111,0.0,0,1716226439811,134,0.0


In [83]:



# مقدار پیپ برای جفت ارز (مثلاً EUR/USD برابر با 0.0001)
pip_value = 0.0001
up_threshold = 5 * pip_value
return_threshold = 2.5 * pip_value



# پیدا کردن زمان‌ها و قیمت‌های مورد نظر
def find_significant_moves(df, up_threshold, return_threshold):
    results = []

    for i in range(len(df)):
        start_price = df.loc[i, 'bid']
        max_price = start_price
        min_price = start_price

        for j in range(i + 1, len(df)):
            current_price = df.loc[j, 'bid']
            max_price = max(max_price, current_price)
            min_price = min(min_price, current_price)

            if max_price - start_price > up_threshold and max_price - current_price <= return_threshold:
                results.append({
                    'start_time': df.loc[i, 'time'],
                    'end_time': df.loc[j, 'time'],
                    'start_price': start_price,
                    'end_price': max_price,
                    'price_change': max_price - start_price
                })
                break

            if start_price - min_price > up_threshold and current_price - min_price <= return_threshold:
                results.append({
                    'start_time': df.loc[i, 'time'],
                    'end_time': df.loc[j, 'time'],
                    'start_price': start_price,
                    'end_price': min_price,
                    'price_change': start_price - min_price
                })
                break

    return pd.DataFrame(results)

significant_moves = find_significant_moves(ticks_frame, up_threshold, return_threshold)

print(significant_moves)




               start_time            end_time  start_price  end_price  price_change
0     2024-05-17 08:04:01 2024-05-17 08:42:39      1.26598    1.26547       0.00051
1     2024-05-17 08:04:03 2024-05-17 08:42:42      1.26597    1.26546       0.00051
2     2024-05-17 08:04:03 2024-05-17 08:42:52      1.26595    1.26545       0.00050
3     2024-05-17 08:04:05 2024-05-17 08:42:52      1.26596    1.26545       0.00051
4     2024-05-17 08:04:05 2024-05-17 08:42:42      1.26597    1.26546       0.00051
...                   ...                 ...          ...        ...           ...
89417 2024-05-20 20:59:45 2024-05-20 21:03:02      1.27067    1.27118       0.00051
89418 2024-05-20 20:59:46 2024-05-20 21:03:04      1.27068    1.27119       0.00051
89419 2024-05-20 21:00:04 2024-05-20 21:03:04      1.27068    1.27119       0.00051
89420 2024-05-20 21:00:04 2024-05-20 21:03:02      1.27067    1.27118       0.00051
89421 2024-05-20 21:00:04 2024-05-20 21:03:04      1.27068    1.27119       

In [96]:
ticks_frame

,time,bid,ask,last,volume,time_msc,flags,volume_real
0,2024-05-17 08:04:01,1.26598,1.26598,0.0,0,1715920441320,130,0.0
1,2024-05-17 08:04:03,1.26597,1.26597,0.0,0,1715920443523,134,0.0
2,2024-05-17 08:04:03,1.26595,1.26595,0.0,0,1715920443724,134,0.0
3,2024-05-17 08:04:05,1.26596,1.26596,0.0,0,1715920445326,134,0.0
4,2024-05-17 08:04:05,1.26597,1.26597,0.0,0,1715920445919,134,0.0
...,...,...,...,...,...,...,...,...
89752,2024-05-20 21:03:58,1.27114,1.27114,0.0,0,1716226438812,134,0.0
89753,2024-05-20 21:03:59,1.27111,1.27111,0.0,0,1716226439334,134,0.0
89754,2024-05-20 21:03:59,1.27110,1.27110,0.0,0,1716226439412,134,0.0
89755,2024-05-20 21:03:59,1.27111,1.27111,0.0,0,1716226439811,134,0.0


In [104]:
significant_moves['hour'] = significant_moves['start_time'].dt.hour
significant_moves['minute'] = significant_moves['start_time'].dt.minute

# ایجاد هیستوگرام برای ساعت
fig_hour = px.histogram(significant_moves, x='hour', nbins=24, title='Histogram of Trades by Hour')
fig_hour.show()

# ایجاد هیستوگرام برای دقیقه
fig_minute = px.histogram(significant_moves, x='minute', nbins=60, title='Histogram of Trades by Minute')
fig_minute.show()



In [99]:
significant_moves['delta'] = (significant_moves['end_time'] - significant_moves['start_time']).dt.total_seconds() / 60

In [103]:
significant_moves.sort_values('delta').to_csv('delta_valu.csv' , index= False)

In [ ]:
significant_moves.head(30)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# ایجاد نمودار اصلی

fig = px.line(ticks_frame, y= ticks_frame['ask']  , x='time', title='نمودار اصلی')
# fig.add_trace(go.Scatter(x=rates_frame['time'], y=rates_frame['close'], mode='lines'))
# اضافه کردن نمودار دوم


# # fig.add_trace(go.Scatter(x=rates_frame['time'], y=y_std, mode='lines', name='Standard Deviation'))
# fig.add_trace(go.Scatter(x=ticks_frame['time'], y=ticks_frame['bid'] , mode='lines'))

fig.update_layout(
    height = 900 , 
    width = 2000
)
# نمایش نمودار
fig.show()

In [394]:

def std(symbol  , preod):


        

    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, 1440)
    rates_frame = pd.DataFrame(rates)
    rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
    rates_frame['valu'] = rates_frame['high'] - rates_frame['low']
    rates_frame['std'] = rates_frame['valu'].rolling( window= preod).std() 
    rates_frame['mean'] = rates_frame['valu'].rolling( window= preod).mean()
    rates_frame['mean_price'] =  (rates_frame['high'] + rates_frame['low'])  / 2
    rates_frame['std_'] = (rates_frame['valu'].rolling( window= preod).std() * 100) / rates_frame['mean_price'].rolling( window= preod).mean()
    rates_frame['std_high'] = rates_frame['high'].rolling( window= preod).std() 
    rates_frame['std_low'] = rates_frame['low'].rolling( window= preod).std() 


    rates_frame['mean_price'] =  (rates_frame['high'] + rates_frame['low'])  / 2

    rates_frame['std_high_'] = (rates_frame['high'].rolling( window= preod).std() * 100) / rates_frame['mean_price'].rolling( window= preod).mean()
    rates_frame['std_low_'] = (rates_frame['low'].rolling( window= preod).std() * 100) / rates_frame['mean_price'].rolling( window= preod).mean()


    return rates_frame['std_'].iloc[-1]  , rates_frame['std_high_'].iloc[-1]  , rates_frame['std_low_'].iloc[-1]  
  
    
std_ , std_high , std_low = std('GBPUSD_o' , 3)










0.004139447948554938 0.007140168122196708 0.0030160194941673967


In [404]:
preod = 3

rates = mt5.copy_rates_from_pos('GBPUSD_o', mt5.TIMEFRAME_M1, 0, 2440)
rates_frame = pd.DataFrame(rates)
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
rates_frame['valu'] = rates_frame['high'] - rates_frame['low']
rates_frame['std'] = rates_frame['valu'].rolling( window= preod).std() 
rates_frame['mean'] = rates_frame['valu'].rolling( window= preod).mean()
rates_frame['mean_price'] =  (rates_frame['high'] + rates_frame['low'])  / 2
rates_frame['std_'] = (rates_frame['valu'].rolling( window= preod).std() * 100) / rates_frame['mean_price'].rolling( window= preod).mean()
rates_frame['std_high'] = rates_frame['high'].rolling( window= preod).std() 
rates_frame['std_low'] = rates_frame['low'].rolling( window= preod).std() 


rates_frame['mean_price'] =  (rates_frame['high'] + rates_frame['low'])  / 2

rates_frame['std_high_'] = (rates_frame['high'].rolling( window= preod).std() * 100) / rates_frame['mean_price'].rolling( window= preod).mean()
rates_frame['std_low_'] = (rates_frame['low'].rolling( window= preod).std() * 100) / rates_frame['mean_price'].rolling( window= preod).mean()


# return rates_frame['std_'].iloc[-1]  , rates_frame['std_high_'].iloc[-1]  , rates_frame['std_low_'].iloc[-1]  

In [405]:
rates_frame


,time,open,high,low,close,tick_volume,spread,real_volume,valu,std,mean,mean_price,std_,std_high,std_low,std_high_,std_low_
0,2024-05-10 02:36:00,1.25259,1.25259,1.25258,1.25259,3,0,0,0.00001,NaN,NaN,1.252585,NaN,NaN,NaN,NaN,NaN
1,2024-05-10 02:37:00,1.25259,1.25263,1.25258,1.25262,10,0,0,0.00005,NaN,NaN,1.252605,NaN,NaN,NaN,NaN,NaN
2,2024-05-10 02:38:00,1.25262,1.25263,1.25257,1.25257,13,0,0,0.00006,0.000026,0.000040,1.252600,0.002112,0.000023,0.000006,0.001844,0.000461
3,2024-05-10 02:39:00,1.25258,1.25262,1.25257,1.25260,13,0,0,0.00005,0.000006,0.000053,1.252595,0.000461,0.000006,0.000006,0.000461,0.000461
4,2024-05-10 02:40:00,1.25261,1.25268,1.25261,1.25266,13,0,0,0.00007,0.000010,0.000060,1.252645,0.000798,0.000032,0.000023,0.002566,0.001844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,2024-05-13 19:21:00,1.25662,1.25663,1.25649,1.25655,19,0,0,0.00014,0.000047,0.000087,1.256560,0.003761,0.000030,0.000072,0.002387,0.005757
2436,2024-05-13 19:22:00,1.25655,1.25655,1.25642,1.25643,48,0,0,0.00013,0.000049,0.000107,1.256485,0.003926,0.000057,0.000096,0.004525,0.007647
2437,2024-05-13 19:23:00,1.25643,1.25650,1.25642,1.25650,31,0,0,0.00008,0.000032,0.000117,1.256460,0.002558,0.000066,0.000040,0.005219,0.003216
2438,2024-05-13 19:24:00,1.25651,1.25652,1.25641,1.25650,57,0,0,0.00011,0.000025,0.000107,1.256465,0.002003,0.000025,0.000006,0.002003,0.000460


In [254]:
rates_frame

,time,open,high,low,close,tick_volume,spread,real_volume,valu,std,mean,mean_price,std_,std_high,std_low,std_high_,std_low_
0,2024-05-10 00:44:00,1.25231,1.25235,1.25227,1.25228,33,11,0,0.00008,NaN,NaN,1.252310,NaN,NaN,NaN,NaN,NaN
1,2024-05-10 00:45:00,1.25227,1.25235,1.25225,1.25231,57,11,0,0.00010,NaN,NaN,1.252300,NaN,NaN,NaN,NaN,NaN
2,2024-05-10 00:46:00,1.25232,1.25236,1.25227,1.25229,38,2,0,0.00009,0.000010,NaN,1.252315,0.000799,0.000006,0.000012,0.000461,0.000922
3,2024-05-10 00:47:00,1.25227,1.25234,1.25227,1.25230,36,0,0,0.00007,0.000015,NaN,1.252305,0.001220,0.000010,0.000012,0.000799,0.000922
4,2024-05-10 00:48:00,1.25231,1.25236,1.25228,1.25235,35,0,0,0.00008,0.000010,0.000084,1.252320,0.000799,0.000012,0.000006,0.000922,0.000461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,2024-05-10 23:55:00,1.25238,1.25238,1.25230,1.25238,19,0,0,0.00008,0.000012,0.000058,1.252340,0.000922,0.000000,0.000012,0.000000,0.000922
1392,2024-05-10 23:56:00,1.25236,1.25242,1.25235,1.25235,21,0,0,0.00007,0.000015,0.000072,1.252385,0.001220,0.000023,0.000036,0.001844,0.002879
1393,2024-05-10 23:57:00,1.25235,1.25244,1.25235,1.25244,28,0,0,0.00009,0.000010,0.000088,1.252395,0.000798,0.000031,0.000029,0.002439,0.002305
1394,2024-05-10 23:58:00,1.25243,1.25248,1.25243,1.25245,102,0,0,0.00005,0.000020,0.000078,1.252455,0.001597,0.000031,0.000046,0.002439,0.003688


In [216]:
x = rates_frame['std'] 

# x = rates_frame['valu']* 10000

px.histogram(rates_frame , x = x )

In [401]:
import plotly.express as px
import plotly.graph_objects as go

# ایجاد نمودار اصلی
fig = px.line(rates_frame, y=(rates_frame['open'] +  rates_frame['close']) / 2, x='time', title='نمودار اصلی')
fig = px.line(rates_frame, y=(rates_frame['mean'] ) , x='time', title='نمودار اصلی')
# fig.add_trace(go.Scatter(x=rates_frame['time'], y=rates_frame['close'], mode='lines'))
# اضافه کردن نمودار دوم
y_std =  (rates_frame['open'] +  rates_frame['close']) / 2  - rates_frame['std'] 

y__std =  rates_frame['std'] 

# fig.add_trace(go.Scatter(x=rates_frame['time'], y=y_std, mode='lines', name='Standard Deviation'))
fig.add_trace(go.Scatter(x=rates_frame['time'], y=y__std, mode='lines'))

fig.update_layout(
    height = 900 , 
    width = 2000
)
# نمایش نمودار
fig.show()





In [218]:
px.line(rates_frame , x = 'time' , y = 'std_')

In [219]:
px.line(rates_frame , x = 'time' , y = 'std_high_')

In [402]:
px.line(rates_frame , x = 'time' , y = 'std_')

In [407]:
import plotly.graph_objects as go

# ایجاد نمودار کندل
fig = go.Figure(data=[go.Candlestick(x=rates_frame['time'],
                open=rates_frame['open'],
                high=rates_frame['high'],
                low=rates_frame['low'],
                close=rates_frame['close'])])

# اضافه کردن خط عمودی
threshold =  0.02  #  0.0175 
threshold_dates = rates_frame[rates_frame['std_'] > threshold]['time']
fig.add_trace(go.Scatter(x=threshold_dates, y=[max(rates_frame['high'])]*len(threshold_dates),
                         mode='markers', marker=dict(color='red', symbol='line-ns-open')))

fig.update_layout(
    height=900,
    width=2000
)

# نمایش نمودار
fig.show()

print(len(threshold_dates))



7


In [259]:
import datetime
import pytz

# تعیین منطقه‌ی زمانی تهران
tehran_timezone = pytz.timezone('Asia/Tehran')

# گرفتن زمان فعلی در تهران
tehran_now = datetime.datetime.now(tehran_timezone)

print(tehran_now)


3
